![alt text](../../pythonexposed-high-resolution-logo-black.jpg "Optionele titel")

# NumPy Extra's: Handige Technieken

### Topic 1: Multidimensionale Arrays Maken Uit Bestanden

Wanneer je een NumPy-array maakt, maak je een sterk geoptimaliseerde datastructuur. Een van de redenen hiervoor is dat een NumPy-array **alle elementen in een aaneengesloten geheugenruimte opslaat**. Deze geheugentechniek betekent dat de gegevens in hetzelfde geheugenzone worden opgeslagen, waardoor de toegangstijden snel zijn. Dit is uiteraard zeer wenselijk.  

Er ontstaat echter een probleem wanneer je je array moet *uitbreiden*.

Stel dat je **meerdere bestanden moet importeren in een multidimensionale array**.  

Je kunt ze in aparte arrays lezen en ze vervolgens samenvoegen met `np.concatenate()`. 
> Dit zou echter een *kopie van je oorspronkelijke array* maken voordat de kopie met de extra gegevens wordt uitgebreid.
> Het kopieerproces is nodig om ervoor te zorgen dat de bijgewerkte array nog steeds aaneengesloten in het geheugen bestaat, omdat de oorspronkelijke array mogelijk niet-gerelateerde inhoud naast zich had.

Telkens opnieuw arrays kopiëren wanneer je nieuwe gegevens uit een bestand toevoegt, kan het verwerken vertragen en is inefficiënt voor het geheugen van je systeem. Het probleem wordt erger naarmate je meer gegevens aan je array toevoegt. **Hoewel dit kopieerproces is ingebouwd in NumPy, kun je de effecten ervan minimaliseren met deze twee stappen:**

1. *Bepaal bij het opzetten van je initiële array hoe groot deze moet zijn voordat je deze vult.* Je kunt zelfs overwegen om de grootte te overschatten om toekomstige gegevensuitbreidingen te ondersteunen. Als je eenmaal deze afmetingen kent, kun je je array van tevoren maken.

2. *De tweede stap is om deze te vullen met de brongegevens*. Deze gegevens worden *in je bestaande array* geplaatst zonder dat deze uitgebreid hoeft te worden.

### Invoeren van data in de Arrays

In dit eerste voorbeeld gebruik je gegevens uit drie bestanden om een driedimensionale array te vullen. De inhoud van elk bestand wordt hieronder getoond, en je vindt deze bestanden ook in het downloadbare materiaal:

Het eerste bestand heeft twee rijen en drie kolommen met de volgende inhoud:

**CSV file1.csv**:
```
1.1, 1.2, 1.3
1.4, 1.5, 1.6
```
Het tweede bestand heeft dezelfde afmetingen en bevat het volgende:
**CSV file2.csv**:
```
2.1, 2.2, 2.3
2.4, 2.5, 2.6
```
Het derde bestand, ook met dezelfde afmetingen, bevat deze cijfers:
**CSV file3.csv**:
```
3.1, 3.2, 3.3
3.4, 3.5, 3.6
```

Voordat je verdergaat, voeg je deze drie bestanden toe aan je programmamap. 

De resulterende NumPy-array die je maakt van de drie bestanden zal er als volgt uitzien:

![alt text](resulterende_array.png "Optionele titel")

Je ziet dat `file1.csv` de voorste laag van je array vormt, `file2.csv` het middelste deel en `file3.csv` de achterste laag.  

De code om deze array te maken is de volgende:

In [4]:
import numpy as np
from pathlib import Path

# Bepaal de huidige werkdirectory en zoek naar de gewenste bestanden
# glob(): Dit is een methode die bestanden en mappen in een directory zoekt die voldoen aan een bepaald patroon. 
# Het retourneert een generator met de paden naar de gevonden bestanden.
# "file?.csv" is een glob-patroon:
#     file: De bestandsnaam begint met "file"
#     ?: Dit is een wildcard die precies één willekeurig teken vervangt. Bijvoorbeeld file1.csv of fileA.csv, maar niet file10.csv (dat bevat meer dan één teken).
#     .csv: De bestanden moeten eindigen met de extensie .csv
# .glob() geeft de files niet altijd in de verwachte volgorde -> sorted!
csv_files = sorted(Path.cwd().glob("file?.csv"))

# Maak een lege array van de juiste vorm (aantal bestanden, rijen, kolommen)
# Hier aangenomen dat elk bestand dezelfde vorm heeft, bijvoorbeeld 2x3
num_files = len(csv_files)
array_shape = (num_files, 2, 3)  # Pas de vorm aan afhankelijk van je data
array = np.zeros(array_shape)

# Lees de bestanden en vul de array
# numpy.loadtxt() in NumPy wordt gebruikt om gegevens uit een tekstbestand in te lezen en om te zetten in een NumPy-array
for file_count, csv_file in enumerate(csv_files):
    array[file_count] = np.loadtxt(csv_file, delimiter=",")

print("Gecombineerde array:")
print(array)

Gecombineerde array:
[[[1.1 1.2 1.3]
  [1.4 1.5 1.6]]

 [[2.1 2.2 2.3]
  [2.4 2.5 2.6]]

 [[3.1 3.2 3.3]
  [3.4 3.5 3.6]]]


### Wat met verschillende sizes van data?

**Undersized data**

Om te beginnen voeg je enkele te kleine gegevens toe. Je vindt deze in een bestand genaamd short_file.csv, dat slechts één rij bevat: CSV short_file.csv 4.1, 4.2, 4.3

Je wilt dit aan het einde van je array toevoegen zoals hieronder getoond:

![alt text](updated_array.png "Optionele titel")

In [25]:
# Inlezen van de 3 initiële files en 
array = np.zeros((4, 2, 3))
for file_count, csv_file in enumerate(sorted(Path.cwd().glob("file?.csv"))):
    array[file_count] = np.loadtxt(csv_file.name, delimiter=",")

array[3, 0] = np.loadtxt("short_file.csv", delimiter=",")
array

array([[[1.1, 1.2, 1.3],
        [1.4, 1.5, 1.6]],

       [[2.1, 2.2, 2.3],
        [2.4, 2.5, 2.6]],

       [[3.1, 3.2, 3.3],
        [3.4, 3.5, 3.6]],

       [[4.1, 4.2, 4.3],
        [0. , 0. , 0. ]]])

#### Inlezen:
- **Vier bestanden inlezen**: Dit keer worden vier aparte bestanden ingelezen.
- **Aanvankelijke Array Vorm**: De array die je maakt heeft de vorm **(4, 2, 3)**.
- **Extra Dimensie Nodig**:
  - Om het vierde bestand op te slaan, voeg je een extra dimensie toe langs **as-0**.
  - Dit gebeurt in **regel 1**.

#### Gebruik van de For-Lus:
- **Eerste Drie Bestanden**:
  - De **for-lus** leest de eerste drie bestanden, net zoals eerder.
- **Kort Bestand Inlezen**:
  - Python moet exact weten **waar** je het vierde (korte) bestand wilt plaatsen.
  - Gebruik **array[3, 0]**:
    - **3** staat voor de indexpositie langs **as-2**.
    - **0** specificeert dat de invoer vanaf rij **0** moet starten.

#### Efficiëntie:
- **Geheugengebruik**:
  - De array werd vooraf aangemaakt.
  - Dit voorkomt dat het object meerdere keren inefficiënt gekopieerd hoeft te worden.

**Oversized data**  

Stel dat het vierde bestand in plaats van te kort juist te lang was. Je vraagt je misschien af hoe je met zulke bestanden moet omgaan. Deze keer gebruik je een bestand genaamd long_file.csv, dat een extra rij bevat.  Je wil dit document opnemen in je array op de positie zoals hieronder aangegeven. Zoals je in het diagram kunt zien, moet de rest van de array worden uitgebreid met een extra rij om het te kunnen accommoderen.

![alt text](array_extended.png "Optionele titel")

In [15]:
# Create an initial 4x2x3 array filled with zeros
array = np.zeros((4, 2, 3))
print("Initial array ID:", id(array))

# Load CSV files into the array
for file_count, csv_file in enumerate(sorted(Path.cwd().glob("file?.csv"))):
    array[file_count] = np.loadtxt(csv_file.name, delimiter=",")

# Insert a new row filled with zeros into the array at index position 2 along the chosen axis
array = np.insert(arr=array, obj=2, values=0, axis=1)

# Load data from "long_file.csv" into the 3rd row of the array
array[3] = np.loadtxt("long_file.csv", delimiter=",")

# Print the ID of the updated array
print("Updated array ID:", id(array))

# Print the final array
print("Final array:\n", array)

Initial array ID: 128168773750672
Updated array ID: 128168773749904
Final array:
 [[[1.1 1.2 1.3]
  [1.4 1.5 1.6]
  [0.  0.  0. ]]

 [[2.1 2.2 2.3]
  [2.4 2.5 2.6]
  [0.  0.  0. ]]

 [[3.1 3.2 3.3]
  [3.4 3.5 3.6]
  [0.  0.  0. ]]

 [[4.1 4.2 4.3]
  [4.4 4.5 4.6]
  [4.7 4.8 4.9]]]


### Topic 2: Tabulaire gegevens in Gestructureerde Arrays

Met de NumPy-arrays die je in het vorige voorbeeld hebt gemaakt, was er geen manier om de betekenis van de gegevens in elke kolom te weten. Het zou mooi zijn als je specifieke kolommen kon verwijzen met betekenisvolle namen in plaats van indexnummers. In plaats van bijvoorbeeld `student_grades = results[:, 1]` te gebruiken, zou je `student_grades = results["exam_grade"]` kunnen gebruiken. Goed nieuws! Je kunt dit doen door een gestructureerde array te maken.

Gestructureerde arrays in NumPy zijn een type array waarmee je kunt werken met tabulaire gegevens, vergelijkbaar met een spreadsheet of een database tabel. In plaats van dat alle elementen hetzelfde type moeten hebben, zoals in standaard NumPy-arrays, kun je verschillende datavelden definiëren, elk met zijn eigen type.

### Waarom gestructureerde arrays gebruiken?
1. **Tabulaire gegevens:**
   - Ideaal voor het opslaan van datasets waarin rijen meerdere kolommen met verschillende typen bevatten, zoals een lijst van personen met namen (strings), leeftijden (integers), en inkomens (floats).
   
2. **Flexibiliteit:**
   - Elk veld heeft zijn eigen naam, wat zorgt voor een duidelijke en gestructureerde toegang tot data.

3. **Efficiëntie:**
   - Data wordt opgeslagen in een enkel blok geheugen, wat het efficiënt maakt voor grote datasets.

### Hoe maak je een gestructureerde array?

#### 1. **Met een dtype specificatie**
Je definieert een dtype met veldnamen en hun corresponderende typen.

```python
import numpy as np

# Definieer een dtype
dtype = [('naam', 'U10'), ('leeftijd', 'i4'), ('inkomen', 'f8')]

# Maak een gestructureerde array
data = np.array([('Alice', 25, 50000.0),
                 ('Bob', 30, 60000.0),
                 ('Charlie', 35, 70000.0)], dtype=dtype)

print(data)
```

**Output:**
```plaintext
[('Alice', 25, 50000.) ('Bob', 30, 60000.) ('Charlie', 35, 70000.)]
```

#### 2. **Data benaderen**
Je kunt data benaderen met veldnamen:

```python
# Toegang tot een veld
print(data['naam'])       # Output: ['Alice' 'Bob' 'Charlie']
print(data['leeftijd'])   # Output: [25 30 35]

# Specifieke rij
print(data[1])            # Output: ('Bob', 30, 60000.)
```

#### 3. **Data toevoegen of wijzigen**
```python
data['leeftijd'][1] = 32  # Wijzig de leeftijd van Bob
print(data)

# Output:
# [('Alice', 25, 50000.) ('Bob', 32, 60000.) ('Charlie', 35, 70000.)]
```

### Meer geavanceerde mogelijkheden

#### 1. **Complexere dtypes**
Je kunt veldtypen combineren, zoals lijsten, tuples of geneste velden:

```python
dtype = [('id', 'i4'), ('scores', 'f4', (3,))]  # Een veld 'scores' met 3 floats
data = np.array([(1, [95.0, 88.0, 92.0]),
                 (2, [78.0, 81.0, 85.0])], dtype=dtype)

print(data)
print(data['scores'])  # Output: [[95. 88. 92.] [78. 81. 85.]]
```

#### 2. **Velden sorteren**
```python
sorted_data = np.sort(data, order='leeftijd')
print(sorted_data)
```

#### 3. **Filtreren**
Je kunt eenvoudig filters toepassen op een veld:

```python
high_income = data[data['inkomen'] > 55000]
print(high_income)  # Output: [('Bob', 32, 60000.) ('Charlie', 35, 70000.)]
```

### Alternatieven
Voor complexe datastructuren en gemengde types wordt vaak **pandas** gebruikt in plaats van gestructureerde arrays, omdat pandas veel meer functionaliteit biedt voor data-analyse en manipulatie.

### Wanneer gestructureerde arrays gebruiken?
1. **Als efficiëntie belangrijk is:**
   - Gestructureerde arrays zijn efficiënter dan Python-datastructuren zoals lijsten of dictionaries.
2. **Als je een eenvoudige tabulaire dataset hebt:**
   - Bij meer complexe datasets (met ontbrekende waarden, geavanceerde indexen, etc.) is pandas meestal beter geschikt.


### Overzicht van veelgebruikte Numpy Datatypes:

| **Categorie**            | **Datatype**                 | **Beschrijving**                                                                                       |
|--------------------------|------------------------------|--------------------------------------------------------------------------------------------------------|
| **Numerieke datatypes**  | `int8`, `int16`, `int32`, `int64`  | Gehele getallen (positief/negatief) met respectievelijk 1, 2, 4 of 8 bytes.                               |
|                          | `uint8`, `uint16`, `uint32`, `uint64` | Unsigned integers (alleen positief) met respectievelijk 1, 2, 4 of 8 bytes.                              |
|                          | `float16`, `float32`, `float64` | Zwevendekommagetallen met respectievelijk 16, 32 of 64 bits precisie.                                  |
|                          | `complex64`, `complex128`    | Complexe getallen met zwevendekommagetallen van respectievelijk 32 bits en 64 bits.                    |
| **Tekst- en stringtypen**| `U` (Unicode)               | Unicode-strings, bijvoorbeeld `U10` (Unicode met maximaal 10 tekens).                                  |
|                          | `S` (Bytestring)            | Bytestrings, bijvoorbeeld `S10` (bytestring met maximaal 10 bytes).                                    |
| **Boolean**              | `bool`                      | Voorwaarden die `True` of `False` representeren.                                                      |
| **Datetimetypes**        | `datetime64`                | Datums en tijden, bijvoorbeeld `datetime64[D]` (dag), `datetime64[ms]` (milliseconden).               |
|                          | `timedelta64`               | Tijdsverschillen of intervallen.                                                                       |
| **Geavanceerde structuren** | Gestructureerde arrays    | Combinatie van types, zoals `dtype=[('veld1', 'int32'), ('veld2', 'float64')]`.                        |
|                          | `object`                    | Algemene objecttypen, bijvoorbeeld Python-objecten. Minder efficiënt dan andere datatypes.             |



### Gestructureerde Arrays Maken: voorbeeld

Een gestructureerde array is een NumPy-array met een datatype dat bestaat uit een set tuples, elk met een veldnaam en een regulier datatype. Zodra je deze hebt gedefinieerd, kun je elk afzonderlijk veld benaderen en wijzigen met behulp van de veldnaam.

Het onderstaande voorbeeld laat zien hoe je een gestructureerde NumPy-array maakt en verwijst:

```python
race_results = np.array([
    ("At The Back", 1.2, 3),
    ("Fast Eddie", 1.3, 1),
    ("Almost There", 1.1, 2),
], dtype=[("horse_name", "U12"), ("price", "f4"), ("position", "i4")])

print(race_results["horse_name"])
```

Het gestructureerde array is gedefinieerd met drie velden: `horse_name`, `price` en `position`. Hiermee kun je de gegevens eenvoudiger benaderen.

### Topic 3: Dubbele Gegevens Verwijderen Uit een NumPy-Array

Het komt vaak voor dat je gegevens hebt met duplicaten, die je wilt verwijderen om analyses of verdere verwerking eenvoudiger te maken. NumPy biedt hiervoor handige methoden om dubbele rijen of waarden uit een array te verwijderen.

Stel dat je een array hebt met enkele dubbele waarden, zoals hieronder weergegeven:

```python
import numpy as np

# Maak een array met dubbele waarden
data = np.array([1, 2, 2, 3, 4, 4, 5])

# Verwijder dubbele waarden met np.unique()
unieke_data = np.unique(data)

print(unieke_data)
```

De uitvoer van dit script is:

```
[1 2 3 4 5]
```
Met de functie `np.unique()` kun je eenvoudig alle dubbele waarden verwijderen, zodat je alleen de unieke elementen overhoudt. Dit is vooral handig bij het werken met grote datasets waarin duplicaten de analyse kunnen verstoren.

Als je met tweedimensionale arrays werkt en je wilt duplicaten op rijniveau verwijderen, kun je `np.unique()` gebruiken met de parameter `axis=0` om ervoor te zorgen dat hele rijen als duplicaten worden beschouwd:

```python
# Maak een 2D-array met dubbele rijen
data_2d = np.array([[1, 2], [3, 4], [1, 2], [5, 6], [3, 4]])

# Verwijder dubbele rijen
unieke_data_2d = np.unique(data_2d, axis=0)

print(unieke_data_2d)
```

De uitvoer van dit script is:

```
[[1 2]
 [3 4]
 [5 6]]
```

Hiermee worden alle dubbele rijen uit de array verwijderd, zodat je alleen de unieke rijen overhoudt.

### Topic 4: Specifieke Delen van Hiërarchische Gegevens Analyseren 

Hiërarchische gegevens zijn gegevens die uit verschillende niveaus bestaan, waarbij elk niveau is gekoppeld aan de niveaus direct boven en onder. In dit voorbeeld analyseren we een aandelenportefeuille met meerdere investeringen, waarbij elke investering dagelijkse prijsbewegingen bevat.

Stel dat je een gestructureerde array hebt met de dagelijkse prijzen van verschillende bedrijven, zoals hieronder weergegeven:

In [19]:
import numpy as np

portfolio = np.array([
    ('Company A', 'Technology', 100.5, 101.2, 102.0, 101.8, 112.5),
    ('Company B', 'Finance', 200.1, 199.8, 200.5, 201.0, 200.8),
    ('Company C', 'Healthcare', 50.3, 50.5, 51.0, 50.8, 51.2),
    ('Company D', 'Technology', 110.5, 101.2, 102.0, 111.8, 97.5),
], dtype=[('company', 'U20'), ('sector', 'U20'), ('mon', 'f8'), ('tue', 'f8'), ('wed', 'f8'), ('thu', 'f8'), ('fri', 'f8')])

In [20]:
# Filter bedrijven in de technologiesector
tech_mask = portfolio['sector'] == 'Technology'
tech_companies = portfolio[tech_mask]

In [21]:
tech_companies

array([('Company A', 'Technology', 100.5, 101.2, 102., 101.8, 112.5),
       ('Company D', 'Technology', 110.5, 101.2, 102., 111.8,  97.5)],
      dtype=[('company', '<U20'), ('sector', '<U20'), ('mon', '<f8'), ('tue', '<f8'), ('wed', '<f8'), ('thu', '<f8'), ('fri', '<f8')])

In [22]:
# Haal de vrijdagprijzen van technologiebedrijven op
friday_prices = tech_companies['fri']
companies = tech_companies['company']

In [23]:
companies

array(['Company A', 'Company D'], dtype='<U20')

In dit voorbeeld hebben we een array `portfolio` met informatie over verschillende bedrijven en hun dagelijkse prijzen. We hebben gefilterd op bedrijven in de technologiesector en de vrijdagprijzen opgehaald. Dit helpt om inzicht te krijgen in de prestaties van specifieke delen van de gegevens.

### Topic 5: Je Eigen Gevectoriseerde Functies Schrijven

Een van de voordelen van NumPy is het vermogen om berekeningen uit te voeren op volledige arrays zonder dat de programmeur langzame loops hoeft te schrijven die door elke rij of elk element gaan. In plaats daarvan gebruikt NumPy de onderliggende C-taal om de berekening op de hele array uit te voeren. Dit staat bekend als vectorisatie.

In deze laatste sectie werk je met het bestand full_portfolio.csv

De onderstaande code laat vectorisatie in actie zien:

In [16]:
share_dtypes = [
    ("company", "U20"),
    ("sector", "U20"),
    ("mon", "f8"),
    ("tue", "f8"),
    ("wed", "f8"),
    ("thu", "f8"),
    ("fri", "f8"),
]

portfolio = np.loadtxt(
    Path("full_portfolio.csv"),
    delimiter=",",
    dtype=share_dtypes,
    skiprows=1,
)

portfolio["fri"] - portfolio["mon"]

array([ 12. ,   0.7,   0.9, -13. ,   0.7,  -3.1])

Na het construeren van de gestructureerde array portfolio, besluit je te kijken hoe je aandelen het gedurende de week hebben gedaan. Om dit te doen, kies je twee arrays — een met de aandelenprijzen van maandag en een met die van vrijdag. Om de wekelijkse verandering te zien, trek je de maandagprijzen af van de vrijdagprijzen.

> Let op dat, hoewel je één array van de andere aftrekt, NumPy elk individueel element van de arrays aftrekt zonder dat je code hoeft te schrijven die door elk van hen afzonderlijk loopt. Dit is vectorisatie.

**Een eigen functie vectoriseren**  

Stel nu dat je een extra bonus van 10% krijgt op aandelen die gedurende de week die je analyseert met meer dan 1% in waarde zijn gestegen. Om je winst inclusief de bonus te berekenen, moet je rekening houden met twee gevallen — die aandelen die de bonus krijgen en die niet. Hiervoor zou je het volgende kunnen proberen:

In [17]:
def profit_with_bonus(first_day, last_day):
    if last_day >= first_day * 1.01:
        return (last_day - first_day) * 1.1
    else:
        return last_day - first_day

profit_with_bonus(portfolio["mon"], portfolio["fri"])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

Hier krijg je een ValueError omdat de functie niet in staat is de arrays die je hebt doorgegeven te interpreteren. De functie werkt alleen als je twee getallen doorgeeft. Om deze functie te laten werken met arrays, kun je gebruikmaken van np.vectorize().

In [18]:
vectorized_profit_with_bonus = np.vectorize(profit_with_bonus)
vectorized_profit_with_bonus(portfolio["mon"], portfolio["fri"])

array([ 13.2 ,   0.7 ,   0.99, -13.  ,   0.7 ,  -3.1 ])

Dit verandert de oorspronkelijke functie in een versie die met arrays werkt en toont aan dat je de winst inclusief bonus kunt berekenen voor alle aandelen.